# Heroes of Pymoli

The Heoroes of Pymoli is a free-to-play game that encourages players to purchase items to enhance their playing experience.  Using the data gathered from the software, we came up with the following analysis

+ There are about 573 registered players of which 81% are males.
+ Players between the ages of 20-24 purchase more of the optional items.
+ Item prices average about $2.83.
+ Lower priced items are more popular than higher priced items.
+ The game's audience tends to lean to teenagers (15-19) and young adults (20-24).
+ The most profitale items are not considered the most popular items.

In [1]:
import pandas as pd
import numpy as np

# open json file
data_file = "Resources/purchase_data.json"
# data_file_pd = pd.read_json(data_file)
data_file_pd = pd.read_json(data_file)

In [2]:
# read columns
data_file_pd.columns

Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object')

## Player Count

In [3]:
# total number of uniques players
totalplayers= data_file_pd['SN'].nunique()
playerCount = pd.DataFrame([totalplayers], columns=['Total Players'])
playerCount.style.format({"Total Players": "{:,}"})

,Total Players
0,573


## Purchasing Analysis (Total)

In [4]:
# Total number of unique items
totalItems = data_file_pd['Item ID'].nunique()

# Total number of Purchases
totalPurchase = data_file_pd['Price'].count()

# Average purchase price
averagePrice = data_file_pd['Price'].mean()

# Total Revenue
totalRevenue = round(data_file_pd['Price'].sum(), 2)




totalAnalysis_df = pd.DataFrame([[totalItems,averagePrice,totalPurchase,totalRevenue]], columns=['Number of Unique Items','Average Price','Number of Purchases','Total Revenue'])
totalAnalysis_df.style.format({"Number of Purchase": "{:,}","Number of Unique Items": "{:,}","Average Price": "${:,.2f}", "Total Revenue": "${:,.2f}"})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


## Gender Demographics

In [5]:
# pd.options.display.float_format = '{:,.2f}'.format
uniqueSN =pd.DataFrame (data_file_pd['SN'].unique(), columns=['SN'])
gender = data_file_pd.groupby(['Gender'])['SN'].nunique()
gender_df = pd.DataFrame(index=["Female", "Male", "Other / Non-Disclosed"])
gender

Gender
Female                   100
Male                     465
Other / Non-Disclosed      8
Name: SN, dtype: int64

In [6]:
percent = data_file_pd.groupby(['Gender'])['SN'].nunique()/totalplayers * 100
# percent

In [7]:
gender_df["Percentage of Players"] = percent
# gender_df

In [8]:
gender_df['Total Count'] = gender
gender_df.style.format({"Total Count": "{:,}","Percentage of Players": "{:,.2f}"})

,Percentage of Players,Total Count
Female,17.45,100
Male,81.15,465
Other / Non-Disclosed,1.40,8


## Purchasing Analysis (Gender)

In [9]:
# find the Price count, mean, sum, normalized totals by Gender
genderGroup_df = data_file_pd.groupby(['Gender'])

purchaseCount = genderGroup_df["Price"].count()
averagePrice = round(genderGroup_df["Price"].mean(), 2)
totalPurchase = genderGroup_df["Price"].sum()
normalizedTotals = round(totalPurchase/genderGroup_df["SN"].nunique(), 2)

genderPurchase_df = pd.DataFrame({
    'Purchase Count': purchaseCount,
    'Average Purchase Price': averagePrice,
    'Total Purchase Value': totalPurchase,
    'Normalized Totals': normalizedTotals
})

# rearrange the columns
genderPurchase_final = genderPurchase_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
genderPurchase_final.style.format({"Total Revenue": "{:,}","Number of Unique Items": "{:,}","Average Purchase": "{:,.2f}%", "Total Revenue": "${:,.2f}"})
genderPurchase_final.style.format({"Purchase Count": "{:,}","Normalized Totals": "${:,.2f}","Average Purchase Price": "${:,.2f}","Total Purchase Value": "${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


## Age Demographics

In [10]:
# setup bins and labels
agebins =[0,10,15,20,25,30,35,40,data_file_pd['Age'].max()+1]
binlabels = [" <10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

# create an age bin
purchase_data = data_file_pd.copy()
purchase_data['AgeBins'] =pd.cut(data_file_pd["Age"], bins=agebins,labels=binlabels,include_lowest=True, right=False)

In [11]:
playerCount = purchase_data.groupby(['AgeBins'])['SN'].nunique()
percentPlayer = round((playerCount/totalplayers)*100, 2)

ageDemographics_final = pd.DataFrame({
    "Total Count": playerCount,
    "Percentage of Players": percentPlayer
})

ageDemographics_final.style.format({"Percentage of Players": "{:,.2f}"})


,Percentage of Players,Total Count
AgeBins,,
<10,3.32,19
10-14,4.01,23
15-19,17.45,100
20-24,45.20,259
25-29,15.18,87
30-34,8.20,47
35-39,4.71,27
40+,1.92,11


## Purchase Analysis (Age)

In [12]:
priceCount = purchase_data.groupby(['AgeBins'])['Price'].count()
priceMean = round(purchase_data.groupby(['AgeBins'])['Price'].mean(), 2)
priceSum = round(purchase_data.groupby(['AgeBins'])['Price'].sum(), 2)
normalizedTotals = round((priceSum / purchase_data.groupby(['AgeBins'])['SN'].nunique()), 2)

Purchase_Price_df = pd.DataFrame({
    "Purchase Count": priceCount,
    "Average Purchase Price" : priceMean,
    "Total Purchase Value" : priceSum,
    "Normalized Totals" : normalizedTotals
})

# Final report
Purchase_By__Age_final = (Purchase_Price_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value","Normalized Totals"]])
Purchase_By__Age_final.style.format({"Normalized Totals": "${:,.2f}","Average Purchase Price": "${:,.2f}","Total Purchase Value": "${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
AgeBins,,,,
<10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89


## Top Spenders

In [13]:
# find the count, average, and total by player
priceCount = data_file_pd.groupby(['SN'])['Price'].count()
priceAverage = round(data_file_pd.groupby(['SN'])['Price'].mean(), 2)
priceTotal = round(data_file_pd.groupby(['SN'])['Price'].sum(), 2)
# priceTotal

topSpender_df = pd.DataFrame({
    "Purchase Count" : priceCount,
    "Average Purchase Price" : priceAverage,
    "Total Purchase Value" : priceTotal
})

# List top 5 spenders in game
topSpender_final = (topSpender_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]).sort_values("Total Purchase Value", ascending=False).head(5)
topSpender_final.style.format({"Average Purchase Price": "${:,.2f}","Total Purchase Value": "${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


## Most Popular Items

In [14]:
# Find count, average, and total by Item
itemCount = data_file_pd.groupby(['Item ID','Item Name'])['Price'].count()
itemPrice = round(data_file_pd.groupby(['Item ID','Item Name'])['Price'].max(), 2)
itemTotal = round(data_file_pd.groupby(['Item ID','Item Name'])['Price'].sum(), 2)

popularItems_df = pd.DataFrame({
    "Purchase Count" : itemCount,
    "Item Price" : itemPrice,
    "Total Purchase Value" : itemTotal
})

# 5 most popular item
popularItems_final = (popularItems_df[["Purchase Count", "Item Price", "Total Purchase Value"]]).sort_values("Purchase Count", ascending=False).head(5)
popularItems_final.style.format({"Item Price": "${:,.2f}","Total Purchase Value": "${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


## Most Profitable Items

In [15]:
# 5 most profitable item
popularItems_final = (popularItems_df[["Purchase Count", "Item Price", "Total Purchase Value"]]).sort_values("Total Purchase Value", ascending=False).head(5)
popularItems_final.style.format({"Item Price": "${:,.2f}","Total Purchase Value": "${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
